<h1>Data Processing</h1>

Ziel dieses Notebooks ist, die gescrapeten Daten so zu verarbeiten, dass Sie später thematisch durchsucht werden können. Ein Embeddingvektor liefert genau diese Funktion. Auf eine Anfrage hin wird der Abstand zwischen dem Embedding der Frage und den Embeddings aller anderen Dokumenten berechnet. Dann werden die Dokumente mit den kleinsten Abständen ausgewählt und dem LLM als Kontext mitgegeben. Mithilfe des Wissens dieser Dokumente soll dass LLM dann in der Lage sein die Frage korrekt zu beantworten.
Für die Embeddings benutzen wir [Google Bert](https://blog.google/products/search/search-language-understanding-bert/)

Beispiel:

User: Welche Dozenten unterrichten das Fach Grundlagen der Informatik?

System wählt besten 5 Dokumente aus 

    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...
    

Aus der Nutzeranfrage und den Dokumenten wird eine neue Query erstellt, die dem LLM dann final bereitgetellt wird. Diese sieht in etwa so aus:

    
    <Dokument 1>: ... betreute Prof. Dr. Löhr eine Batchelorarbeit in Grundlagen der     Informatik...
    <Dokument 2>: Prof. Dr. Weber tel.: 013882664 email: weber@th.de Raum: HQ: 403, Fächer: Grundlagen der Informatik ...
    <Dokument 3> ...
    <Dokument 4> ...
    <Dokument 5> ...

    Bitte beantworte folgende Frage unter der Berücksichtigung obiger Dokumente:
    Welche Dozenten unterrichten das Fach Grundlagen der Informatik?


Das LLM wird daraufhin hoffentlich korrekt eine Antwort liefern die ähnlich ist zu:

    A: An der TH Nürnberg Georg Simon Ohm unterichten die Professoren Prof. Dr. Löhr und Prof. Dr. Weber das Fach Grundlagen der Informatik.


In [ ]:
import sqlite3
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
from scipy.spatial.distance import cosine
from db_init import db_get_df, db_save_df
import json

c:\Users\lizab\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

Zunächst laden wir die Daten aus der Datenbank. Dabei besitzt jedes Dokument als Metadaten den Titel der Webseite, den filenamen und den Text. Diese speichern wir uns in einen Pandas Dataframe

In [ ]:
df = db_get_df("html_attrs", ["filename", "title", "text"])

print(df.dtypes)
print(df["text"][3])

Zur überprüfung der Texte können wir nun einmal eine Keywordsuche starten. Dieser Ansatz wird außerdem tiefer im Notebook [spacy_keywordextraction](./spacy_keywordextraction.ipynb) verfolgt.

In [ ]:
word = "Gallwitz"

[text for text in df["text"] if word in text][:5]

Jetzt werden wir für jedes Dokument ein eigenes Word embeddings erstellen. Dazu müssen wir zunächst das BERT Model laden.
Das BERT Model ist ein von Google trainiertes mehrschichtiges neuronales Netz, welches ursprünglich dafür entwickelt wurde, dass ???
es ist trainiert auf 10.000+ Büchern
es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur vorbereitung die zusätzlichen Token
[SEP] um das Ende eines Satzes zu markieren
[CLS] am Anfang des Texten
[PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


In [ ]:
model = BertModel.from_pretrained('bert-base-uncased') #TODO try better model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

testSentence = "In der Bibliothek gibt es 40 Bücher zu Thema Animes"
tokens_question = tokenizer.tokenize(testSentence)
tokens_question


In [ ]:
df["tokens"] = [tokenizer.tokenize(text) for text in df["text"]]

In [ ]:
print(df.dtypes)
print(df["tokens"][2])

In [ ]:
tokens_question = ['[CLS]'] + tokens_question + ['[SEP]']
attention_mask = [1 if token != "[PAD]" else 0  for token in tokens_question]
token_ids = tokenizer.convert_tokens_to_ids(tokens_question)
print(token_ids)

In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)

attention_mask = torch.tensor(attention_mask).unsqueeze(0)

In [ ]:
output = model(token_ids, attention_mask=attention_mask)

In [ ]:
print(output[2])

In [ ]:
output[0].shape

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)

In [ ]:
def proccessSentence(tokens):
    if len(tokens) == 0:
        # Handle the case when the token list is empty, for example, return a default embedding or raise an exception.
        # For demonstration purposes, we'll return a zero tensor as the default embedding.
        return torch.zeros(768)

    # Ensure the token sequence length is no longer than the maximum sequence length the model can handle (512)
    if len(tokens) > 512:
        tokens = tokens[:512]

    # Padding the token sequence to the maximum sequence length if it's shorter
    if len(tokens) < 512:
        tokens += ['[PAD]'] * (512 - len(tokens))

    attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]
    tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokens)
    tokensDocument_tensor = torch.tensor([tokenDocument_idss], dtype=torch.int64)
    segmentsDocument_tensors = torch.tensor([attention_mask], dtype=torch.int64)

    with torch.no_grad():
        outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
        hiddenDocuments_states = outputs[2]

    tokenDocuments_vecs = hiddenDocuments_states[-2][0]
    sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
    #print("Our final sentence embedding vector of shape:", sentenceDocument_embedding)

    return sentenceDocument_embedding

df["word_embeddings"] = [proccessSentence(tokens).tolist() for tokens in tqdm(df["tokens"])]


In [ ]:
import ast
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE 
import numpy as np  

word_embeddings_copy = df['word_embeddings'].copy()

# Verwenden Sie 'ast.literal_eval', um Zeichenketten in Listen umzuwandeln
word_embeddings_copy = word_embeddings_copy.apply(ast.literal_eval)

# Wandeln Sie die Listen von Listen in ein Numpy-Array um
word_embeddings = np.array(word_embeddings_copy.tolist())

tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
X_embedded = tsne.fit_transform(word_embeddings)
plt.figure(figsize=(10, 6))
plt.scatter(X_embedded[:, 0], X_embedded[:, 1], s=5)
plt.title("t-SNE Visualization of Word Embeddings")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.show() 


In [ ]:
# database = 'discord_bot/scrap/html.sqlite'

# with sqlite3.connect(database) as con:
#     html_df.to_sql('html_with_embeddings', con, index=False, if_exists='replace')

In [ ]:
#dokument--> bert anwenden für jeden dokument
# question-bert anwenden
question="was macht Gallwitz?"
document=df["text"][1]

tokens_question = tokenizer.tokenize(question)
tokens_document = tokenizer.tokenize(document)
attetion_mask_question = [1] * len(tokens_question)
attention_mask_dokument = [1] * len(tokens_document)

token_idss = tokenizer.convert_tokens_to_ids(tokens_question)
tokenDocument_idss = tokenizer.convert_tokens_to_ids(tokens_document)


tokens_tensor = torch.tensor([token_idss])
segments_tensors = torch.tensor([attetion_mask_question])

tokensDocument_tensor = torch.tensor([tokenDocument_idss])
segmentsDocument_tensors = torch.tensor([attention_mask_dokument])

with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

with torch.no_grad():
    outputs = model(tokensDocument_tensor, segmentsDocument_tensors)
    hiddenDocuments_states = outputs[2]

# print(token_idss)
# print(tokenDocument_idss)

print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))


Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 6
Number of hidden units: 768


In [ ]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_vecs_sum = []

# For each token in the sentence...
for token in token_embeddings:
    
    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)
    
    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

In [ ]:
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


tokenDocuments_vecs = hiddenDocuments_states[-2][0]

# Calculate the average of all 22 token vectors.
sentenceDocument_embedding = torch.mean(tokenDocuments_vecs, dim=0)
print ("Our final sentence embedding vector of shape:", sentence_embedding)


In [ ]:


# Calculate the cosine similarity between the word bank 
# in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(sentence_embedding, sentenceDocument_embedding)

print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

In [ ]:
database = 'html.sqlite'
sql = """
SELECT filename, title, text, word_embeddings FROM word_embeddings
"""

con = sqlite3.connect(database)
df = pd.read_sql_query(sql, con)
con.close()

print(df.dtypes)
print(df["text"][3])



In [ ]:
import json

print(len(df["text"][4]))
# print(json.loads(df["text"][0]))
diff_bank = 1 - cosine(json.loads(df["word_embeddings"][3]), json.loads(df["word_embeddings"][6]))
# for embed in df["word_embeddings"]:
# print(embed)

print(diff_bank)

In [ ]:
df["text"][1746]

In [ ]:
word = "Fachhochschulgesetz"

df.loc[df["text"].str.contains(word)]["text"]
# [text for text in df["text"] if word in text][:5]

In [ ]:
import sys
sys.path.insert(0, '/Users/br/Projects/IT-Ptojekt-Chatbot/daibl/discord_bot')
from question_embedding import question_embeddings
import json
import matplotlib.pyplot as plt


# TODO 10 Fragen
# TODO TSNE

df = db_get_df("chunk_word_embeddings_all")
print(df.dtypes)
question = "TH"
question_embedding = question_embeddings(question)

df["distance"] = [1 - cosine(json.loads(embedding), question_embedding) for embedding in df["chunk_word_embeddings"]]
most_similar_documents = df.nsmallest(5, "distance")
print(f"question embedding: {question_embedding[:10]}")
print(most_similar_documents["distance"])


df["distance"].plot(kind='hist', bins=200)
plt.show()

In [ ]:
df = db_get_df("word_embeddings", ["filename", "title", "text", "tokens"])
df["token_ids"] = [tokenizer.convert_tokens_to_ids(json.loads(tokens)) for tokens in df["tokens"]]

In [ ]:
# splice dokuments in 512 token chunks

# Initialize an empty list to store rows for the new DataFrame
new_rows = []

# Function to split text and tokens into chunks of 512 tokens
def split_text_and_tokens(row):
    text = row['text']
    tokens_ids = row['token_ids']
    filename = row['filename']

    if len(tokens_ids) > 512:
        # Split into multiple chunks
        for i in range(0, len(tokens_ids), 512):
            chunk_tokens = tokens_ids[i:i + 512]
            chunk_text = tokenizer.decode(chunk_tokens)

            # Create a new row with a reference to the original row
            new_row = {'filename': filename, 'chunk_id': i/512, 'chunk_text': chunk_text, 'chunk_tokens_json': json.dumps(chunk_tokens)}
            new_rows.append(new_row)
    else:
        # If the row has 512 tokens or fewer, keep it as is
        new_row = {'filename': filename, 'chunk_id': 0, 'chunk_text': text, 'chunk_tokens_json': json.dumps(tokens_ids) }
        new_rows.append(new_row)

# Apply the function to each row in the original DataFrame
df.apply(split_text_and_tokens, axis=1)

# Create a new DataFrame from the list of new rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame if needed
new_df.reset_index(drop=True, inplace=True)

# Print the new DataFrame
print(new_df.to_markdown())

# tokenizer.convert_tokens_to_string


Aus den 2433 Dokumenten die wir eigentlich gescraped haben, sind nun 6945 chunks entstanden es hat sich fast verdreifacht. Wenn man die 787 Seiten ohne Inhalt abzieht, hat sich die Anzahl von 1646 auf 6158 fast vervierfacht.

In [ ]:
new_df["chunk_tokens_json"][2]

In [ ]:
db_save_df(new_df, "chunk_word_embeddings")

In [ ]:
import sqlite3
from dotenv import load_dotenv
import os

load_dotenv()
database_path = os.getenv("DATABASE_PATH")

def merge_db_tables():
    # Connect to your SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()
    # Create the new table using the structure of the first table (chunk_word_embeddings_0)
    cursor.execute('''CREATE TABLE chunk_word_embeddings_all AS SELECT * FROM chunk_word_embeddings_0 WHERE 0''')
    # Insert data from the other tables into the new table
    for i in range(1, 8):
        cursor.execute(f'INSERT INTO chunk_word_embeddings_all SELECT * FROM chunk_word_embeddings_{i}')
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

merge_db_tables()